<strong>Preliminary Analysis</strong> <br>

 Your preliminary analysis should include: <br>
<ul>
  <li><strong>Data Cleaning Code </strong> <br>
        Code for cleaning and processing your data. Include a data dictionary for your transformed dataset. 
    <ul> 
      <li>
      Data Dictionary:
        <ul>
            <li><strong>id</strong>: A unique identifier for each tweet</li>
            <li><strong>text</strong>: the text of the tweet</li>
            <li><strong>location</strong>: the location the tweet was sent</li>
            <li><strong>keyword</strong>: a particular keyword from the tweet</li>
            <li><strong>target</strong>: denotes whether a tweet is about a real disaster(1) or not(0)</li>
        </ul>
      </li> 
    </ul> 
  </li>
  <li><strong>Exploratory Analysis </strong> <br>
       Describe what work you have done so far and include the code. This may include descriptive statistics, graphs and charts, and preliminary models. 
    <ul> 
      <li>
        For this preliminary analysis, we noticed that there were inconsistencies in the data columns so we adjusted them using the steps in the next bullet point.
        <li>
          The <strong>keyword</strong> column did not extract all the disaster keywords from the text. So we extracted the keyword column and text column and applied a function to extract the correct keywords by word tokenizing and extracting the words which intersect with the disaster word vector. 
        <li>
          The same was done for the <strong>location</strong> column.
        </li>
        </li>
      </li> 
    </ul> 
  </li>
  <li><strong>Challenges </strong> <br>
        Describe any challenges you've encountered so far. Let me know if there's anything you need help with! 
    <ul> 
      <li>
        There were many challenges when attempting to correct the inconsistent data columns. 
      </li> 
      <li>
          The <strong>keyword</strong> column wasn't as difficult but still came as a challenge as attempting the fillna using normal means wasnt working for us. But, we figured it out. 
      </li> 
      <li>
        The <strong>location</strong> column was difficult. Hands down took the longest to implement and even now doesnt work correctly as there arent efficient and free NER libraries to extract 'GPE' from texts. We attempted using 3 different types, one of which included using <strong>spaCy</strong>, but that didnt work with Google Colab. The other two that we used were very time consuming to run. Took a whomping <strong>17~20 minutes</strong> when using on the train data set. 
      </li> 
    </ul> 
  </li>
  <li><strong>Future Work </strong> <br>
        Describe what work you are planning to complete for the final analysis.
    <ul> 
      <li>Future work involves using our cleaned data and features as input for models suited for classification, like Naive Bayes and Logisitic Regression and training these models</li> 
      <li>Making predictions off our trained models and evaluating performance with accuracy scores and confusion matrices</li>
      <li>Defining our grid of hyperparameter values and using GridSearchCV() to systematically find the best peforming model</li>
      </ul> 
  </li>
  <li><strong>Contributions </strong> <br>
        Describe the contributions that each group member made. 
    <ul> 
      <li>
      Daniel Hernandez
      <ul> 
        <li>Found the data sets. 
        </li>
        <li>Helped in brain storming. 
        </li> 
        <li>Created colab template
        </li>
        <li>Implemented visualizations
        </li>
        <li>Assisted in thinking of procedure to clean data columns.
        </li>
    </ul> 
      </li> 
      <li>
      Justin Park
      <ul> 
        <li>Helped in brain storming.
        </li>
        <li>Implemented functions to clean keyword and location columns.
        </li> 
        <li>Implemented procedure to clean data columns.
        </li>
    </ul> 
      </li> 
    </ul> 
  </li>
</ul>
 Your preliminary analysis does not have to be a formal writeup. You will submit your Github repository for your project.




Notes for 11/17

- How do we deal with empty values for Keyword and Location?
- How would those columns be inputted into our model?
- Are these columns worth adding to the model in the first place or should we omit them?
- Should we normalize our tweets?
- How should we tokenize our Tweets?
<br>
<br>
<h2>Notes for 11/19 </h2>
<strong>Data cleaning </strong>
<ul>
<li>clean location feature, there is dirty data such as dates, symbols, and numeric values. </li>
<li>fill in NA values with numpy's NaN for machine learning. </li>
<li>set up word vector for emergencies.</li>
<li>use word vector to fill in NA values in the keyword section from text. </li>
</ul>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import Tree
import spacy
# import matplotlib
# matplotlib.use('Agg')
!pip install locationtagger
import locationtagger
import re


In [2]:
import nltk
# essential entity models downloads
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

nlp = spacy.load('en')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
!pip install git+git://github.com/jmbielec/geograpy3.git 
!pip install git+https://github.com/jmbielec/geograpy3.git 
import geograpy3

  Cloning git://github.com/jmbielec/geograpy3.git to /tmp/pip-req-build-9s22t7im
  Running command git clone -q git://github.com/jmbielec/geograpy3.git /tmp/pip-req-build-9s22t7im
  Cloning https://github.com/jmbielec/geograpy3.git to /tmp/pip-req-build-e2d9uig8
  Running command git clone -q https://github.com/jmbielec/geograpy3.git /tmp/pip-req-build-e2d9uig8
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_dat

In [4]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# First lets get a general idea of what our data looks like:

In [5]:
# Count the amount of rows and columns
print("Shape of train_df:", train_df.shape)
print('\n\n')
# Counting the amount of missing values
print("null values:\n", train_df.isnull().sum())
print('\n\n')
# Observing the DataFrame's summary
train_df.info()

Shape of train_df: (7613, 5)



null values:
 id             0
keyword       61
location    2533
text           0
target         0
dtype: int64



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [6]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1



*   It looks like we need to fill in some values for keyword and location
*   Lets see if we can fill in these values based on what we find in the 'text' values

<h2>Fill in missing values for keyword column </h2>

In [7]:
# create word vector which contains the dissaster keywords
dissaster_vector = set(train_df['keyword'].unique());
# function which will fill in missing values for keyword column dependent on text column 
def fillKey(text):
  # tokenize the text and store into text_vector
  text_vector = set(word_tokenize(text.lower()))
  # find the dissaster keywords in text_vector
  filler = list(set.intersection(dissaster_vector, text_vector))
  # combine the filler array with a delimiter matching that of original format from keyword column
  filler = '%20'.join(filler)
  return filler

In [8]:
# auxilery DataFrame to handle DataFrame.apply()
t_df = pd.DataFrame()
t_df['keyword'] = train_df['text'].copy()
t_df.head()

,keyword
0,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...


In [9]:
# fill missing values for keyword column
t_df['keyword'] = t_df['keyword'].apply(fillKey)

# check if it worked
print(t_df.head(), '\n\n')
print(t_df.isnull().sum())

      keyword
0  earthquake
1        fire
2  evacuation
3  evacuation
4       smoke 


keyword    0
dtype: int64


In [10]:
# before replace missing
display(train_df.head())

# replacing the missing keyword values
train_df['keyword'] = t_df['keyword']

# after replace missing
display(train_df.head())

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


,id,keyword,location,text,target
0,1,earthquake,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,fire,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,evacuation,NaN,All residents asked to 'shelter in place' are ...,1
3,6,evacuation,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,smoke,NaN,Just got sent this photo from Ruby #Alaska as ...,1


<h2>Now to fill in missing values for locations </h2>

In [11]:
def fillTagLoc(txt):
  txt = re.sub(r'[#|@]',r'',txt)
  place_entity = locationtagger.find_locations(text = txt)
  filler = '%20'.join(set(place_entity.countries + place_entity.regions + place_entity.cities))
  return filler

In [12]:
def fillGeoLoc(text_input):
  text_input = re.sub(r'[#|@]',r'',text_input)
  place_entity = geograpy3.get_place_context(text = text_input)
  filler = '%20'.join(set(place_entity.countries + place_entity.regions + place_entity.cities))
  return filler

In [13]:
test_str = train_df['text'][5]

print(test_str, '\n\n')

# testing the fillLoc function
print('fillTagLoc:\n',fillTagLoc(test_str), '\n\n' )
print('fillGeoLoc:\n',fillGeoLoc(test_str), '\n\n' )

#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires 


fillTagLoc:
  


place_entity:  <geograpy3.places.PlaceContext object at 0x7ff39f2a9950>
fillGeoLoc:
 Update%20CAfire%20RockyFire%20Lake County 




In [ ]:
# now that we've gotten keyword column missing values filled in where it could, time to do same for location. 
# auxilery DataFrame to handle DataFrame.apply()
l_df = pd.DataFrame()
l_df['location'] = train_df['text'].copy()
l_df.head()

# fill missing values for keyword column.... ** WARNING !!! this took 17m to execute!
l_df['location'] = l_df['location'].apply(fillTagLoc)


In [ ]:
# check if it worked
print(l_df.head(), '\n\n')
print(l_df.isnull().sum())

In [ ]:
# Great! it worked~~~~~~ 
# before 
display(train_df.head(10))
# now to fill in the missing values 
train_df['location'].fillna(l_df['location'], inplace=True)
# after
display(train_df.head(10))

In [ ]:
# locations using spaCy 
# using .apply via PandaSeries instead of DataFrame
# location_df = train_df["text"].apply(nlp)

# print(location_df.ents)
# test_df.loc[df['keyword'] == null] = ['ablaze if']